In [117]:
from yaml import load 
from datetime import datetime, timedelta
import requests
from pprint import pprint
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage
import httplib2
import argparse
from apiclient import discovery

In [118]:
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
CLIENT_SECRET_FILE ='client_secret_grants.json'
APPLICATION_NAME ='Google Sheets API Report'

In [119]:
credential_path = 'sheets.googleapis.com-report.json'
store = Storage(credential_path)
credentials = store.get()
parser = argparse.ArgumentParser(formatter_class=argparse.RawDescriptionHelpFormatter, parents=[tools.argparser])
flags = parser.parse_args([])
if not credentials or credentials.invalid:
    flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
    flow.user_agent = APPLICATION_NAME
    if flags:
        credentials = tools.run_flow(flow, store, flags)
    print('Storing credentials to ' + credential_path)

In [120]:
http = credentials.authorize(httplib2.Http())
discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?version=v4')
service = discovery.build('sheets', 'v4', http = http, discoveryServiceUrl = discoveryUrl)

In [121]:
spreadsheetId = '1v_BJppYtDnFTZAIVH_crUMtkbrcShOS1cXCCPYBvb50'
rangeName = 'partners dict!A1:C10'

In [122]:
result = service.spreadsheets().values().get(spreadsheetId = spreadsheetId, range = rangeName).execute()
values = result.get('values', [])
print(values)

[['Партнеры', 'utm_source', 'utm_medium'], ['СМИ2', 'smi2', 'cpm'], ['РБК', 'rbc', 'cpc'], ['Infox', 'infox', 'cpc'], ['Infox', 'infox-action', 'cpa']]


In [123]:
f = open('config.yaml', 'r')
config = load(f)
token = config['token']
f = open('report_params.yaml', 'r')
params = load(f)
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
startDate = (datetime.now() - timedelta(days = params['reports']['dau']['days_ago'])).strftime('%Y-%m-%d')
endDate = (datetime.now() - timedelta(days = 1)).strftime('%Y-%m-%d')
dimensions = params['reports']['dau']['dimensions']
metrics = params['reports']['dau']['metrics']
sortby = params['reports']['dau']['sort']
counter = params['metrika counter']

In [124]:
def make_request(**kwargs):
    return requests.get(API_URL, params = kwargs).json()

In [125]:
def json_handler(data):
    report = []
    for line in data['data']:
        dimensions_list = [x['name'] for x in line['dimensions']]
        metrics_list = line['metrics']
        report.append(dimensions_list + metrics_list)
    return report

In [126]:
response = make_request(date1 = startDate, date2 = endDate, dimensions = dimensions, metrics = metrics, 
                    id = counter, sort = sortby, oauth_token = token, accuracy = 1)
data = json_handler(response)
range_data = params['reports']['dau']['range_name']
range_name = '{}!{}{}:{}{}'.format(range_data['list'], range_data['start'], 2, range_data['end'], len(data) + 1)
body = {'values': data}

In [127]:
service.spreadsheets().values().update( spreadsheetId = spreadsheetId, range = range_name, 
                    valueInputOption = 'USER_ENTERED', body = body ).execute()

{'spreadsheetId': '1v_BJppYtDnFTZAIVH_crUMtkbrcShOS1cXCCPYBvb50',
 'updatedRange': 'daily_metrics!A2:E15',
 'updatedRows': 14,
 'updatedColumns': 5,
 'updatedCells': 70}

In [128]:
new_table = service.spreadsheets().values().get( spreadsheetId = spreadsheetId, range = range_name).execute()


In [129]:

services = new_table.get('values', [])
print(services)

[['2019-01-05', '79', '73', '146', '16,4556962'], ['2019-01-06', '73', '69', '101', '6,84931507'], ['2019-01-07', '74', '73', '113', '21,62162162'], ['2019-01-08', '94', '84', '126', '17,0212766'], ['2019-01-09', '118', '109', '196', '14,40677966'], ['2019-01-10', '120', '113', '199', '14,16666667'], ['2019-01-11', '139', '127', '212', '16,54676259'], ['2019-01-12', '139', '117', '224', '18,70503597'], ['2019-01-13', '155', '139', '274', '19,35483871'], ['2019-01-14', '186', '176', '264', '16,66666667'], ['2019-01-15', '165', '155', '271', '13,33333333'], ['2019-01-16', '167', '150', '292', '17,36526946'], ['2019-01-17', '180', '165', '300', '18,33333333'], ['2019-01-18', '175', '169', '337', '13,71428571']]


In [131]:
import pandas as pd
rows = services[1:]

In [132]:
data = pd.DataFrame.from_records(rows)

In [133]:
data

,0,1,2,3,4
0,2019-01-06,73,69,101,"6,84931507"
1,2019-01-07,74,73,113,"21,62162162"
2,2019-01-08,94,84,126,"17,0212766"
3,2019-01-09,118,109,196,"14,40677966"
4,2019-01-10,120,113,199,"14,16666667"
5,2019-01-11,139,127,212,"16,54676259"
6,2019-01-12,139,117,224,"18,70503597"
7,2019-01-13,155,139,274,"19,35483871"
8,2019-01-14,186,176,264,"16,66666667"
9,2019-01-15,165,155,271,"13,33333333"
